<a href="https://colab.research.google.com/github/MRyo-ie/DataMining_Report3_Lv4_NLP_Models/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Report3

## Level 4: 文書分類せよ。


In [6]:
# 自分の Google Drive をマウント
# https://qiita.com/uni-3/items/201aaa2708260cc790b8#drive内のディレクトリをマウントする220180920
from google.colab import drive
import os
drive.mount('/content/gdrive')

#@markdown ### GitHubリポジトリをcloneして保存するパス （Google Drive上）
REPOSITORY_ROOT_DIR = 'Colab Notebooks/learning/old_nlp_techs' #@param {type: "string"}

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### << 実行手順 >>

1.  ↑ の、このボタンを押す　→　　![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)

1. ↖︎（左上）のどこかに「ドライブに保存」があるので、それをクリックして、  
  自分の Google Drive にこのファイルをコピーする。

1. ↑ の変数を設定する。
  - GitHub リポジトリをクローンする場所（GoogleDrive上の好きな場所）のパスを入力。

1. 実行。  
  - ↑で GoogleDrive をマウントするので、（英語の）指示に従って、  
    「ログイン」→「パスコードをコピー」→「四角枠に貼り付けしてEnter」

1. 後は勝手に実行してくれるはず

### 【準備】 パスなどの設定

In [7]:
REPOSITORY_URL = 'https://github.com/MRyo-ie/DataMining_Report3_Lv4_NLP_Models.git'
DATA_DL_URL = "https://www.rondhuit.com/download/ldcc-20140209.tar.gz"
GDRIVE_ROOT = '/content/gdrive/My Drive/'
REPO_PATH = os.path.join(GDRIVE_ROOT, REPOSITORY_ROOT_DIR, REPOSITORY_URL.split('/')[-1][:-4])

# よく使うパス
DATA_DIR = os.path.join(REPO_PATH, 'datas', 'articles_livedoor')
TMP_DIR = os.path.join(DATA_DIR, 'tmp')
FILE_PATH = os.path.join(DATA_DIR, 'tmp', DATA_DL_URL.split('/')[-1])

print("""
------------------------------------------------
<<<  パス一覧  >>>
REPO_PATH = {}

DATA_DL_URL = {}
DATA_DIR = {}
TMP_DIR = {}
FILE_PATH = {}
------------------------------------------------""".format(REPO_PATH, DATA_DL_URL, DATA_DIR, TMP_DIR, FILE_PATH))


------------------------------------------------
<<<  パス一覧  >>>
REPO_PATH = /content/gdrive/My Drive/Colab Notebooks/learning/old_nlp_techs/DataMining_Report3_Lv4_NLP_Models

DATA_DL_URL = https://www.rondhuit.com/download/ldcc-20140209.tar.gz
DATA_DIR = /content/gdrive/My Drive/Colab Notebooks/learning/old_nlp_techs/DataMining_Report3_Lv4_NLP_Models/datas/articles_livedoor
TMP_DIR = /content/gdrive/My Drive/Colab Notebooks/learning/old_nlp_techs/DataMining_Report3_Lv4_NLP_Models/datas/articles_livedoor/tmp
FILE_PATH = /content/gdrive/My Drive/Colab Notebooks/learning/old_nlp_techs/DataMining_Report3_Lv4_NLP_Models/datas/articles_livedoor/tmp/ldcc-20140209.tar.gz
------------------------------------------------


### 【準備】 GitHub clone

In [5]:
if not os.path.exists(REPO_PATH):
    repo_parent_path = os.path.join(GDRIVE_ROOT, REPOSITORY_ROOT_DIR)
    %cd  "$repo_parent_path"
    !git clone "$REPOSITORY_URL"  "$REPO_PATH"
    %cd "$REPO_PATH"
else:
    # pull -f （と同じ意味のコマンド）
    %cd "$REPO_PATH"
    # 最新の状態に更新（強制）
    ! git fetch origin master
    ! git reset --hard origin/master

/content/gdrive/My Drive/Colab Notebooks/learning/old_nlp_techs/DataMining_Report3_Lv4_NLP_Models
Host key verification failed.
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.
Checking out files: 100% (10/10), done.
HEAD is now at d030aec Merge branch 'master' into GoogleColab-merge


### 【準備】 データ
今回は livedoor の記事データセットを使用。


In [0]:
import configparser
import glob
import os, sys
import pandas as pd
import subprocess
import tarfile
from tqdm import tqdm
from urllib.request import urlretrieve

#### DL& 読み込み

In [0]:
# !mkdir -p  "$TMP_DIR"
# %cd  "$TMP_DIR"

# if not os.path.exists(FILE_PATH):
#     ! wget  "$DATA_DL_URL"
# # 時間かかる。
# !tar  -zxvf   'ldcc-20140209.tar.gz'
# !ls

In [8]:
categories = [ 
    name for name 
    in os.listdir( os.path.join(TMP_DIR, "text") ) 
    if os.path.isdir( os.path.join(TMP_DIR, "text", name) ) ]

categories = sorted(categories)
categories

['dokujo-tsushin',
 'it-life-hack',
 'kaden-channel',
 'livedoor-homme',
 'movie-enter',
 'peachy',
 'smax',
 'sports-watch',
 'topic-news']

#### データの抽出・整備



- ラベル（記事の種類）一覧

In [9]:
table = str.maketrans({
    '\n': '',
    '\t': '　',
    '\r': '',
})

def extract_txt(filename):
    with open(filename) as text_file:
        # 0: URL, 1: timestamp
        text = text_file.readlines()[2:]
        text = [sentence.strip() for sentence in text]
        text = list(filter(lambda line: line != '', text))
        return ''.join(text)

%cd  "$TMP_DIR/"

/content/gdrive/My Drive/Colab Notebooks/learning/old_nlp_techs/DataMining_Report3_Lv4_NLP_Models/datas/articles_livedoor/tmp


In [10]:
%cd  "$TMP_DIR/"

# 割と時間かかる。
all_text = []
all_label = []
for cat in tqdm(categories):
    files = glob.glob(os.path.join("text", cat, "{}*.txt".format(cat)))
    files = sorted(files)
    body = [ extract_txt(elem).translate(table) for elem in files ]
    label = [cat] * len(body)
    
    all_text.extend(body)
    all_label.extend(label)


  0%|          | 0/9 [00:00<?, ?it/s]

/content/gdrive/My Drive/Colab Notebooks/learning/old_nlp_techs/DataMining_Report3_Lv4_NLP_Models/datas/articles_livedoor/tmp


KeyboardInterrupt: ignored

- 特徴量とラベル

In [0]:
df = pd.DataFrame({'text' : all_text, 'label' : all_label})
df.head()

In [0]:
df = df.sample(frac=1, random_state=23).reset_index(drop=True)
df.head()

#### ファイルに保存 (機械学習モデル用)

In [0]:
extract_dir = os.path.join(TMP_DIR, 'extracts')
SentencePiece_train_dir = os.path.join(SP_MODEL_DIR, 'tmp')
if not os.path.exists(os.path.join(SP_MODEL_DIR, 'tmp')):
    ! mkdir  "$extract_dir"
    ! mkdir  "$SentencePiece_train_dir"

dpaths = {
    'test' : os.path.join(extract_dir, "test.tsv"),
    'val' : os.path.join(extract_dir, "val.tsv"),
    'train' : os.path.join(extract_dir, "train.tsv"),
}

In [25]:
# Sentence Piece 用の raw text データを作成
df.to_csv( os.path.join(SentencePiece_train_dir, "train.txt"), sep=' ', index=False, header=False, columns=['text'] )
! head -3  "$SentencePiece_train_dir"/train.txt

# 言語モデル用の train, test, dev データファイルを作成
df[:len(df) // 5].to_csv( dpaths['test'], sep='\t', index=False )
df[len(df) // 5:len(df)*2 // 5].to_csv( dpaths['val'], sep='\t', index=False )
df[len(df)*2 // 5:].to_csv( dpaths['train'], sep='\t', index=False )
P = dpaths['test']
! head -3  "$P"

NameError: ignored

### Layer1 : 分かち書き（Tokenize）
- 今回は **Sentence Piece** を使ってみる。  
  https://www.madopro.net/entry/sentencepiece_rnn_lm
- 一番上の２つを実行すれば、あとはここからでOK

In [0]:
# モデル関連のパス
## Sentence Piece
SP_MODEL_DIR = os.path.join(REPO_PATH, 'models', 'Tokenizer', 'SentencePiece')
LANGUAGE_ROOT_DIR = os.path.join(REPO_PATH, 'models', 'LanguageModel')

In [9]:
! pip install sentencepiece

     |████████████████████████████████| 1.0MB 4.8MB/s 


In [10]:
%cd  "$SP_MODEL_DIR"
!echo '[Info] Sentence Piece の学習を開始します。'
if not os.path.exists('model'):
    ! mkdir  'model'

# SentencePiece を学習する：モデルを作る。
import sentencepiece as spm
# 5分くらいかかる。（コメントアウト外す）
if not os.path.exists('model/m.model'):
    spm.SentencePieceTrainer.Train('--input=tmp/train.txt --model_prefix=model/m --vocab_size=8000')
print('[Info] 終了！')

/content/gdrive/My Drive/Colab Notebooks/learning/old_nlp_techs/DataMining_Report3_Lv4_NLP_Models/models/Tokenizer/SentencePiece
[Info] Sentence Piece の学習を開始します。
[Info] 終了！


In [11]:
# 学習したSentencePieceモデルを読み込む
sp = spm.SentencePieceProcessor()
sp.Load('model/m.model')
# こんな感じで 分かち書きできる。
print( sp.EncodeAsPieces('吾輩は猫である') )
# print( sp.EncodeAsPieces(df.at[1, 'text']) )

['▁', '吾', '輩', 'は', '猫', 'である']


### Layer2 : Embedding
- 今回は、BoW+TF-IDF、共起行列、XLNet の予定。

#### 事前処理
- Token（単語など） に ID を振る。

In [12]:
sentences = []
with open('tmp/train.txt', 'r', encoding='utf-8') as f:
    sentences = f.read().split('\n')
print(sentences[7])

"ケーブルいらずのスマホバッテリ！情報満載の1日を振り返る【ITフラッシュバック】ThinkPadをNECで生産するニュースからケーブルいらずのバッテリー、その場で動画を共有するテクニック、315gの最軽量スキャナ、孫正義氏のTwitterまで、様々な記事が掲載された。情報満載の1日を振り返ってみよう。■ThinkPadをNECで生産へ 2012年秋より日本で試験生産【デジ通】レノボとNECパーソナルコンピュータは2011年7月より、NECレノボ・ジャパングループの元で運営されている。それから1年が経過し、ユーザーの目に見える部分では、レノボのサポートサービスをNECに委託し、レノボのIdeaCenterにNECのTV技術を用いた製品が発表されるなど、いつかの点でその成果が出始めている。さらに、一部の個人ユーザーに人気のThinkPadを、NECの米沢事業所で試験生産することを発表した。■Dockコネクターに直挿し！iPhoneやiPadにケーブルいらずのバッテリー【イケショップのレア物】今回紹介する「Universal PowerBank 6000 for Smartphone/Tablet」は、これ1台でかなり対応できるスグレモノのモバイルバッテリーなのである。■iPhoneで撮った動画を仲間で共有！その場で動画を共有するテクニック【知っ得！虎の巻】iPhoneを使えば、手軽に動画を撮影できる。プライベートシーンなら、家族やペットの撮影を楽しめる。ビジネスシーンなら、現場の様子を撮影してわかりやすく報告できる。もし、急ぎの報告が必要なら、いったんパソコンに取り込んでサーバーにアップするより、そのままYouTubeにアップして限定公開してみてはいかがだろう。■315gの最軽量スキャナも登場！PCレスでスキャンデータを活用できる「ADS-2500W」スマートフォンやタブレット端末などの普及やクラウドサービスの利用拡大などから、書類をデータ化して活用する機会が増え、ドキュメントスキャナーのニーズが高まっている。そうしたニーズにこたえるために、ブラザーが新製品を投入する。■電子ブックリーダー「kobo Touch」の実機が見られる！楽天が大手書店と連携した理由「お得に購入できる裏技あり！楽天の書籍端末が凄い5つの理由」でお伝えしたように、楽天の電子ブックリーダー「kob

In [13]:
%cd  "$LANGUAGE_ROOT_DIR"

import pre
from tqdm import tqdm

# data_w2i を作る。
data_w2i = {}
for s in tqdm(sentences):
    tknd_s = sp.EncodeAsPieces(s)
    data_w2i = pre.identify_words(tknd_s, data_w2i)

/content/gdrive/My Drive/Colab Notebooks/learning/old_nlp_techs/DataMining_Report3_Lv4_NLP_Models/models/LanguageModel


100%|██████████| 7368/7368 [00:05<00:00, 1295.00it/s]


#### BoW

In [0]:
from Bag_of_Words import BoW

# 文（章）の list を渡すと、それらを BoW に変換した list を返す。
def build_BoW_vecs(raw_sentences, data_w2i):
    bow = BoW(data_w2i)
    print('[Info] Tokenize中...')
    tknzd_sentences = [sp.EncodeAsPieces(s) for s in raw_sentences]
    return np.array([bow.embed_BoW(tknz_s) for tknz_s in tknzd_sentences])

#### co-matrix

### Layer3 : 分類モデル
- 今回は、SVM で行う。

In [0]:
#@title Embedding Layer のアルゴリズムを選ぶ。
Embed_layer = "BoW"  #@param ['BoW', 'co-matrix', 'XLNet']

In [0]:
extract_dir = os.path.join(TMP_DIR, 'extracts')
dpaths = {
    'test' : os.path.join(extract_dir, "test.tsv"),
    'val' : os.path.join(extract_dir, "val.tsv"),
    'train' : os.path.join(extract_dir, "train.tsv"),
}

# P = dpaths['test']
# ! head -3  "$P"

#### train
##### データ読み込み

In [0]:
import numpy as np
import pandas as pd
from sklearn.utils import resample

In [0]:
###  train.tsv から データを読み込み  ###
def get_datas(mode):
    df = pd.read_csv(dpaths[mode], sep='\t')
    return df['text'].values, df['label'].values

In [53]:
raw_datas, raw_labels = get_datas('train')
print(type(raw_datas), len(raw_labels))

<class 'numpy.ndarray'> 4421


In [54]:
# データをシャッフル
raw_datas, raw_labels = resample(raw_datas, raw_labels, n_samples=len(raw_labels))
print("DataNum: ", len(raw_datas), '\n', raw_labels)
print("DataNum: ", len(datas),  '\n', labels)

DataNum:  4421 
 ['sports-watch' 'dokujo-tsushin' 'sports-watch' ... 'dokujo-tsushin'
 'livedoor-homme' 'livedoor-homme']
DataNum:  4421 
 ['peachy' 'it-life-hack' 'kaden-channel' ... 'movie-enter'
 'dokujo-tsushin' 'it-life-hack']


In [64]:
###  ラベルを数値化  ###
label_list = [ 
    name for name 
    in os.listdir( os.path.join(TMP_DIR, "text") ) 
    if os.path.isdir( os.path.join(TMP_DIR, "text", name) ) ]
print(label_list)

# 変換器（data_w2i） を作成
label_w2i = {}
for i, w in enumerate(name_list):
    label_w2i[w] = i
print(label_w2i)

['dokujo-tsushin', 'livedoor-homme', 'kaden-channel', 'smax', 'topic-news', 'peachy', 'movie-enter', 'it-life-hack', 'sports-watch']
{'dokujo-tsushin': 0, 'livedoor-homme': 1, 'kaden-channel': 2, 'smax': 3, 'topic-news': 4, 'peachy': 5, 'movie-enter': 6, 'it-life-hack': 7, 'sports-watch': 8}


In [0]:
def conv_labels(raw_labels, label_w2i:dict):
    return np.array([label_w2i[w] for w in raw_labels])

##### Embedding

In [66]:
# 生の文（章）から、文（章）ベクトルに変換する。
datas = None
if Embed_layer == 'BoW':
    datas = build_BoW_vecs(raw_datas, data_w2i)

print('[Info] 完了！')
print(datas)

[Info] Tokenize中...
[Info] 完了！
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [2. 0. 0. ... 0. 0. 0.]
 ...
 [2. 0. 0. ... 0. 0. 0.]
 [8. 4. 0. ... 0. 0. 0.]
 [3. 0. 0. ... 0. 0. 0.]]


In [68]:
# ラベルを変換
labels = conv_labels(raw_labels, label_w2i)
print(labels)

[8 0 8 ... 0 1 1]


##### 学習

In [0]:
##  パラメータ関係  ##
from sklearn.svm import LinearSVC
# SVC のハイパーパラメータ
svc = LinearSVC(C=1.0)

In [0]:
# Train
print("****START***********************")
svc.fit(data_vecs, labels)

#### SVM で分類

In [0]:
# coding: UTF-8

import matplotlib
matplotlib.use('Agg')
from matplotlib import pylab
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import glob

from sklearn.metrics import confusion_matrix

def read_csv(path):
    df = pd.read_csv(path, index_col=0)
    print(df.columns)
    return np.array(X),np.array(y)

def normalisation(cm):
    new_cm = []
    for line in cm:
        sum_val = sum(line)
        new_array = [float(num)/float(sum_val) for num in line]
        new_cm.append(new_array)
    return new_cm

def plot_confusion_matrix(cm, name_list_org, title):
    name_list = list(map(lambda x: x.replace('_', '\n'), name_list_org))
    print(name_list)
    df = pd.DataFrame(cm, index=name_list, columns=name_list) #データフレーム化

    plt.clf()
    plt.cla()
    fig, ax = plt.subplots(figsize=(12, 9)) 
    print('df : {}'.format(df))
    #ax.set_ylim(len(cm), 0)
    sns.heatmap(df, square=True, cmap='Blues', vmin=0, vmax=1.0, annot=True,)
    pylab.title(title)
    pylab.xlabel('Predict class')
    pylab.ylabel('True class')

    filename = title + "_reslt.png"
    plt.savefig(filename)
    #plt.close()


#------------------------------------------------------------------------------------



# TEST
prediction = svc.predict(datas[:test_num])
# 混同行列（図）で、結果を出力
## plot
cm = confusion_matrix(labels[:test_num],prediction)
print(normalisation(cm))
d_rate_str = " ({}% → l:t=({}:{}))".format(test_rate_p, len(datas) - test_num, test_num)
plot_confusion_matrix(normalisation(cm), name_list, "SEIYU classifer"+d_rate_str)



###  声感情分類  ###
# Labels (Data Directory)
name_list = ["uemura_happy", "uemura_normal", "uemura_angry"]
#name_list = ["tsuchiya_happy", "tsuchiya_normal", "tsuchiya_angry"]
#name_list = ["fujitou_happy", "fujitou_normal", "fujitou_angry"]

# Read Datas
datas,labels = read_mfcc(name_list, base_dir)
# Suffle Datas
# n_samples : Number of samples to generate.
datas,labels = resample(datas, labels, n_samples=len(labels))
print("DataFileNum:" + str(len(datas)))
test_num = int(len(datas) * split_rate_test)

# TRAINING
svc.fit(datas[test_num:], labels[test_num:])

# TEST
prediction = svc.predict(datas[:test_num])
## plot
cm = confusion_matrix(labels[:test_num],prediction)
print(normalisation(cm))
d_rate_str = " ({}% → l:t=({}:{})".format(test_rate_p, len(datas) - test_num, test_num)
plot_confusion_matrix(normalisation(cm), name_list, "SEIYU emotion classifer"+d_rate_str)



### その他

In [0]:
import torch
import torch.nn as nn
from torch.autograd import Variable

import numpy as np
import pandas as pd
import glob
import os
import time
import math

use_gpu = torch.cuda.is_available()
print('use_gpu : ', use_gpu)

In [0]:
# wakati_corpus.txtから一部データを読み込む
class Corpus(object):
    def __init__(self, sp_loaded, dpaths):
        self.sp = sp_loaded

        self.train = self.load_from_file(dpaths['train'])
        self.valid = self.load_from_file(dpaths['val'])
        self.test = self.load_from_file(dpaths['test'])

        self.train = self.assign_ids(self.train)
        self.valid = self.assign_ids(self.valid)
        self.test = self.assign_ids(self.test)
    
    def load_from_file(self, filename):
        with open(filename, 'r', encoding='utf-8') as f:
            return f.read().split('\n')
    
    def assign_ids(self, texts):
        tokens = []
        for text in texts:
            tokens.extend(self.sp.EncodeAsIds(text))
        
        ids = torch.from_numpy(np.array(tokens, dtype=np.int64))
        return ids

In [0]:
# コーパスを実際に読み込む
corpus = Corpus()